In [1]:
import sys
sys.path.append("/sps/lsst/users/ebarroso/crow")
from crow.cluster_modules.shear_profile import *

In [2]:
import pyccl as ccl
hmf = ccl.halos.MassFuncTinker08(mass_def="200c")
cosmo = ccl.Cosmology(
    Omega_c=0.2607,      # Cold dark matter density
    Omega_b=0.04897,     # Baryon density
    h=0.6766,            # Hubble parameter
    sigma8=0.8102,       # Matter fluctuation amplitude
    n_s=0.9665,          # Spectral index
)
cl_delta_sigma = ClusterShearProfile(cosmo, hmf, 4.0, True)

cl_reduced_shear = ClusterShearProfile(cosmo, hmf, 4.0, False)

cl_reduced_shear_interp = ClusterShearProfile(cosmo, hmf, 4.0, False, True)

cl_reduced_shear_interp_mis = ClusterShearProfile(cosmo, hmf, 4.0, False, False)


### Test values of shear profile

In [3]:
import time

start = time.time()
ds_val = cl_delta_sigma.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Delta sigma:", ds_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear.set_beta_parameters(10)
gt_val = cl_reduced_shear.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear exact:", gt_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear_interp.set_beta_parameters(10)
cl_reduced_shear_interp.set_beta_s_interp(1.1, 1.3)
gt_val = cl_reduced_shear_interp.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear interpolated beta:", gt_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear_interp_mis.set_beta_parameters(10)
cl_reduced_shear_interp_mis.set_beta_s_interp(1.1, 1.3)
cl_reduced_shear_interp_mis.set_miscentering(0.2)
gt_val = cl_reduced_shear_interp_mis.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear interpolated beta with miscentering:", gt_val, " | Time:", end - start, "s")


Delta sigma: [1.13451921e+12]  | Time: 0.0029447078704833984 s
Reduced shear exact: [0.00027987]  | Time: 0.10059022903442383 s
Reduced shear interpolated beta: [0.00027987]  | Time: 0.049571990966796875 s
Reduced shear interpolated beta with miscentering: [0.00022393]  | Time: 0.18524599075317383 s


## Test implementation on the recipe

In [4]:
# from crow.recipes.murata_binned_spec_z import MurataBinnedSpecZRecipe
# from crow.cluster_modules.mass_proxy import MurataBinned
# from crow.cluster_modules.kernel import SpectroscopicRedshift
# #from firecrown.models.cluster import ClusterProperty
# from crow.properties import ClusterProperty
# import time

# pivot_mass, pivot_redshift = 14.625862906, 0.6
# mass_distribution = MurataBinned(pivot_mass, pivot_redshift)
# redshift_distribution = SpectroscopicRedshift()

# def run_recipe(label, cluster_theory, mass_proxy_interval1=(1.1, 1.2), mass_proxy_interval2=(0.2, 0.31), redshiftinterval1=(0.9, 1.3), redshiftinterval2=(0.46, 0.76)):
#     print(f"\n{label} Starts here\n{'-'*50}")
#     average_on = ClusterProperty.NONE
#     average_on |= ClusterProperty.DELTASIGMA
#     recipe = MurataBinnedSpecZRecipe(
#         cluster_theory=cluster_theory,
#         redshift_distribution=redshift_distribution,
#         mass_distribution=mass_distribution,
#     )
#     t0 = time.time()
#     val1 = recipe.evaluate_theory_prediction_shear_profile(mass_proxy_interval1, redshiftinterval1, 1.0, 440, average_on)
#     t1 = time.time()
#     print(f"Eval 1: {val1:.3e} | Time: {t1-t0:.4f} s")

#     t2 = time.time()
#     val2 = recipe.evaluate_theory_prediction_shear_profile(mass_proxy_interval2, redshiftinterval2, 4.0, 440, average_on)
#     t3 = time.time()
#     print(f"Eval 2: {val2:.3e} | Time: {t3-t2:.4f} s")
#     t22 = time.time()
#     val2 = recipe.evaluate_theory_prediction_shear_profile_grid(mass_proxy_interval2, redshiftinterval2, 4.0, 440, average_on, mass_p=30, z_p=30)
#     t33 = time.time()
#     print(f"Eval 3 with grid: {val2:.3e} | Time: {t33-t22:.4f} s")
#     t4 = time.time()
#     counts = recipe.evaluate_theory_prediction_counts(mass_proxy_interval2, redshiftinterval2, 440)
#     t5 = time.time() 
#     print(f"Counts: {counts:.3e} Time: {t5-t4:.4f} s")
#     t6 = time.time()   
#     counts = recipe.evaluate_theory_prediction_counts_grid(mass_proxy_interval2, redshiftinterval2, 440, mass_p=20, z_p=20)
#     t7 = time.time()
#     print(f"Counts grid: {counts:.3e} Time: {t7-t6:.4f} s")


# # Run different scenarios
# run_recipe("Delta Sigma", cl_delta_sigma)
# #run_recipe("Tangential Shear", cl_reduced_shear)
# run_recipe("Tangential Shear with Interpolation", cl_reduced_shear_interp)



Delta Sigma Starts here
--------------------------------------------------
Eval 1: 1.442e+16 | Time: 0.7873 s
Eval 2: 4.393e+15 | Time: 0.3914 s
Eval 3 with grid: 4.414e+15 | Time: 2.1996 s
Counts: 1.937e+03 Time: 0.0022 s
Counts grid: 1.805e+03 Time: 0.1046 s

Tangential Shear with Interpolation Starts here
--------------------------------------------------
Eval 1: 3.724e+00 | Time: 0.6195 s
Eval 2: 1.490e+00 | Time: 0.6452 s
Eval 3 with grid: 1.497e+00 | Time: 2.4289 s
Counts: 1.937e+03 Time: 0.0024 s
Counts grid: 1.805e+03 Time: 0.1045 s


In [5]:
from crow.recipes.murata_binned_spec_z_grid import MurataBinnedSpecZRecipeGrid
from crow.cluster_modules.mass_proxy import MurataUnbinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
# from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time

pivot_mass, pivot_redshift = 14.625862906, 0.6
mass_distribution = MurataUnbinned(pivot_mass, pivot_redshift)
redshift_distribution = SpectroscopicRedshift()


def run_recipe(
    label,
    cluster_theory,
    mass_proxy_interval1=(1.1, 1.2),
    mass_proxy_interval2=(0.2, 0.31),
    redshiftinterval1=(0.9, 1.3),
    redshiftinterval2=(0.46, .76),
    sky_area=440.0,
):
    print(f"\n{label} Starts here\n{'-'*50}")

    average_on = ClusterProperty.NONE
    average_on |= ClusterProperty.DELTASIGMA

    recipe = MurataBinnedSpecZRecipeGrid(
        mass_interval=(12.5, 15.5),
        proxy_interval=(0.2, 1.2),
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
    richness_points = 100,
    redshift_points= 100,
    log_mass_points= 100,
    )
    t0 = time.time()
    # Precompute all grids once
    recipe.recompute_all_grids(sky_area, radius_centers=[1.0, 4.0])
    t1 = time.time()
    print("Grid time",t0 - t1)
    # ---- Shear 1 ----
    t0 = time.time()
    val1 = recipe.evaluate_theory_prediction_shear_profile(
        z_edges=redshiftinterval1,
        mass_proxy_edges=mass_proxy_interval1,
        radius_center=1.0,
        average_on=average_on,
    )
    t1 = time.time()
    print(f"Eval 1 (radius 1.0): {val1:.3e} | Time: {t1 - t0:.4f} s")

    # ---- Shear 2 ----
    t2 = time.time()
    val2 = recipe.evaluate_theory_prediction_shear_profile(
        z_edges=redshiftinterval2,
        mass_proxy_edges=mass_proxy_interval2,
        radius_center=4.0,
        average_on=average_on,
    )
    t3 = time.time()
    print(f"Eval 2 (radius 4.0): {val2:.3e} | Time: {t3 - t2:.4f} s")

    # ---- Counts ----
    t4 = time.time()
    counts = recipe.evaluate_theory_prediction_counts(
        z_edges=redshiftinterval2,
        mass_proxy_edges=mass_proxy_interval2,
        average_on=average_on,
    )
    t5 = time.time()
    print(f"Counts: {counts:.3e} | Time: {t5 - t4:.4f} s")


# Run different scenarios
run_recipe("Delta Sigma", cl_delta_sigma)
#run_recipe("Tangential Shear", cl_reduced_shear)
run_recipe("Tangential Shear with Interpolation", cl_reduced_shear_interp)



Delta Sigma Starts here
--------------------------------------------------
Grid time -9.324459075927734
Eval 1 (radius 1.0): 1.492e+16 | Time: 0.0103 s
Eval 2 (radius 4.0): 1.870e+16 | Time: 0.0105 s
Counts: 1.683e+04 | Time: 0.0088 s

Tangential Shear with Interpolation Starts here
--------------------------------------------------
Grid time -14.4829421043396
Eval 1 (radius 1.0): 6.395e+00 | Time: 0.0113 s
Eval 2 (radius 4.0): 7.033e+00 | Time: 0.0075 s
Counts: 1.683e+04 | Time: 0.0060 s


In [6]:
from crow.recipes.murata_binned_spec_z_grid import MurataBinnedSpecZRecipeGrid
from crow.cluster_modules.mass_proxy import MurataUnbinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
# from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time

pivot_mass, pivot_redshift = 14.625862906, 0.6
mass_distribution = MurataUnbinned(pivot_mass, pivot_redshift)
redshift_distribution = SpectroscopicRedshift()


cluster_theory = cl_delta_sigma
mass_proxy_interval1=(1.1, 1.2)
mass_proxy_interval2=(0.2, 0.31)
redshiftinterval1=(0.9, 1.3)
redshiftinterval2=(0.46, .76)
sky_area=440.0

average_on = ClusterProperty.NONE
average_on |= ClusterProperty.DELTASIGMA

recipe = MurataBinnedSpecZRecipeGrid(
    mass_interval=(12.5, 15.5),
    proxy_interval=(0.2, 1.2),
    cluster_theory=cluster_theory,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution,
richness_points = 100,
redshift_points= 100,
log_mass_points= 100,
)
t0 = time.time()
# Precompute all grids once
recipe.recompute_all_grids(sky_area, radius_centers=[1.0, 4.0])
t1 = time.time()



In [9]:
m_list = recipe.log_mass_points
z_list = recipe.z_points
for mass,z in zip(m_list,z_list):
    hmf = cluster_theory.comoving_volume(np.array([z]), 440) * cluster_theory.mass_function(np.array([mass]), np.array([z]))
    z_index = np.where(z_list == z)[0][0] 
    
    # Find the index of the current mass in the full m_list (N_m)
    mass_index = np.where(m_list == mass)[0][0]
    recipe_hmf = recipe._hmf_grid[z_index, mass_index]
    

1470.5364046062175 [1470.53640461]
8518.056128945018 [8518.05612895]
20146.156497096614 [20146.1564971]
35255.79645807263 [35255.79645807]
52827.140124154255 [52827.14012415]
72012.74410486226 [72012.74410486]
92095.89138559517 [92095.8913856]
112475.67365037341 [112475.67365037]
132653.04447442963 [132653.04447443]
152219.10539047548 [152219.10539048]
170843.5852037273 [170843.58520373]
188264.94496621029 [188264.94496621]
204281.77612768178 [204281.77612768]
218744.62798594031 [218744.62798594]
231549.12657080794 [231549.12657081]
242629.83992838315 [242629.83992838]
251954.92553336127 [251954.92553336]
259521.25926436926 [259521.25926437]
265349.62703024875 [265349.62703025]
269481.11424729094 [269481.11424729]
271940.19339368807 [271940.19339369]
272752.9467650311 [272752.94676503]
272079.51256766607 [272079.51256767]
270013.5237045584 [270013.52370456]
266655.9572172694 [266655.95721727]
262112.09797006095 [262112.09797006]
256490.3258643508 [256490.32586435]
249900.83262291856 [2